In [ ]:
""" 
This file generates feature_extraction_inception_v3
because this is a time consuming operation, 
feature_extraction_inception_v3 can be downloaded from internet.
"""
import os
import sys
import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging
import pprint
import _pickle as cPickle # for python 3
import numpy as np

model_file = "./data/inception_v3_graph_def.pb"
input_description_file = "./data/results_20130124.token"
input_img_dir = "./data/flickr30k_images/"  
output_folder = "./data/feature_extraction_inception_v3"  # get from flickr30K_images,

batch_size = 100

if not gfile.Exists(output_folder):
    gfile.MakeDirs(output_folder)


def parse_token_file(token_file):
    """Parses token file into a dict with 
    key: img_name, value: a list of descriptions"""
    img_name_to_tokens = {}
    with gfile.GFile(token_file, 'r') as f:
        lines = f.readlines()
    for line in lines:
        img_id, description = line.strip('\r\n').split('\t')
        img_name, _ = img_id.split('#')
        img_name_to_tokens.setdefault(img_name, [])
        img_name_to_tokens[img_name].append(description)
    return img_name_to_tokens

img_name_to_tokens = parse_token_file(input_description_file)
all_img_names = img_name_to_tokens.keys()

In [ ]:
pprint.pprint(img_name_to_tokens)
logging.info("num of all images: %d" % len(all_img_names))
pprint.pprint(list(img_name_to_tokens.keys())[0:5])
pprint.pprint(img_name_to_tokens['2778832101.jpg'])

In [ ]:
def load_pretrained_inception_v3(model_file):
    """get v3 graph"""
    with gfile.FastGFile(model_file, "rb") as f:
        graph_def = tf.GraphDef()  # empty graph
        graph_def.ParseFromString(f.read())  # read v3 graph
        _ = tf.import_graph_def(graph_def, name="")  # put v3 graph to default graph
load_pretrained_inception_v3(model_file)

In [ ]:
"""Run  this graph"""
num_batches = int(len(all_img_names) / batch_size)
if len(all_img_names) % batch_size != 0:
    num_batches += 1

with tf.Session() as sess:
    # we use second to last layer as the ouput(feature)  1*1*2048
    second_to_last_tensor = sess.graph.get_tensor_by_name("pool_3:0")
    
    for i in range(num_batches):
        batch_img_names = all_img_names[i*batch_size: (i+1)*batch_size]
        batch_features = []
        for img_name in batch_img_names:
            img_path = os.path.join(input_img_dir, img_name)
            logging.info("processing img %s" % img_name)
            if not gfile.Exists(img_path):
                raise Exception("%s doesn't exists" % img_path)
                
                
            img_data = gfile.FastGFile(img_path, "rb").read()
            feature_vector = sess.run(second_to_last_tensor,
                                      feed_dict = {"DecodeJpeg/contents:0": img_data})
            batch_features.append(feature_vector)
            
        batch_features = np.vstack(batch_features)
        output_filename = os.path.join(output_folder, "image_features-%d.pickle" % i)
        logging.info("writing to file %s" % output_filename)
        with gfile.GFile(output_filename, 'w') as f:
            cPickle.dump((batch_img_names, batch_features), f)